## loading data ##

In [1]:
#for access to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls 'drive/MyDrive/machine learning university/Apartments in Germany'

clean_data_x.csv  clean_data_y.csv


In [6]:
#for read and manipulate data
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [8]:
clean_data_x = pd.read_csv('drive/MyDrive/machine learning university/Apartments in Germany/clean_data_x.csv')
clean_data_x.head()

,Unnamed: 0,serviceCharge,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,totalRent,yearConstructed,scoutId,hasKitchen,cellar,yearConstructedRange,baseRent,lift,baseRentRange,geo_plz,noRooms,thermalChar,floor,numberOfFloors,noRoomsRange,garden,livingSpaceRange,regio1_Baden_Württemberg,regio1_Bayern,regio1_Berlin,regio1_Brandenburg,regio1_Bremen,regio1_Hamburg,regio1_Hessen,regio1_Mecklenburg_Vorpommern,regio1_Niedersachsen,regio1_Nordrhein_Westfalen,regio1_Rheinland_Pfalz,regio1_Saarland,regio1_Sachsen,regio1_Sachsen_Anhalt,regio1_Schleswig_Holstein,regio1_Thüringen,...,firingTypes_solar_heating:oil,firingTypes_solar_heating:oil:electricity,firingTypes_solar_heating:oil:wood,firingTypes_solar_heating:pellet_heating,firingTypes_solar_heating:pellet_heating:bio_energy,firingTypes_solar_heating:pellet_heating:gas,firingTypes_solar_heating:wood,firingTypes_solar_heating:wood:environmental_thermal_energy,firingTypes_solar_heating:wood_chips,firingTypes_steam_district_heating,firingTypes_wind_energy,firingTypes_wood,firingTypes_wood_chips,condition_first_time_use,condition_first_time_use_after_refurbishment,condition_fully_renovated,condition_mint_condition,condition_modernized,condition_need_of_renovation,condition_negotiable,condition_refurbished,condition_ripe_for_demolition,condition_well_kept,interiorQual_luxury,interiorQual_normal,interiorQual_simple,interiorQual_sophisticated,petsAllowed_negotiable,petsAllowed_no,petsAllowed_yes,typeOfFlat_apartment,typeOfFlat_ground_floor,typeOfFlat_half_basement,typeOfFlat_loft,typeOfFlat_maisonette,typeOfFlat_other,typeOfFlat_penthouse,typeOfFlat_raised_ground_floor,typeOfFlat_roof_storey,typeOfFlat_terraced_flat
0,0,0.308220,False,False,-0.591717,0.628648,-1.229879e+00,-2.002236e-03,-0.033579,-0.868944,False,True,-0.705474,-0.005074,False,0.106010,0.251312,0.515956,1.391175e+00,-3.432713e-01,-1.125204e-01,1.523536,True,0.660359,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
1,1,-0.056542,False,True,-0.279627,0.041365,-1.229879e+00,1.060728e-14,-2.287243,0.352699,False,False,-1.116939,0.005419,False,0.557608,1.085543,0.136224,1.341014e-12,1.113710e-13,1.296553e-12,0.456976,False,0.660359,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
2,2,0.341081,True,True,-0.279627,-0.341646,-1.726934e+00,1.301498e-02,1.261079,0.494191,False,True,2.174778,0.013865,True,1.009207,-1.301747,0.136224,1.341014e-12,2.684000e-01,8.408414e-02,0.456976,False,0.660359,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0
3,3,-0.305795,False,True,-0.123581,-0.949357,7.321796e-01,1.060728e-14,-0.057555,0.153687,False,False,-0.705474,-0.017973,False,-0.797187,-0.995898,0.136224,-6.000803e-01,2.684000e-01,1.296553e-12,0.456976,False,-0.760976,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
4,4,-0.043397,False,True,1.436871,-0.474423,-2.156249e-13,5.446935e-05,-0.393207,0.622478,False,False,-1.116939,0.003628,False,0.557608,-0.326283,0.136224,1.547720e+00,-3.432713e-01,1.296553e-12,0.456976,False,0.660359,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0


In [9]:
clean_data_y = pd.read_csv('drive/MyDrive/machine learning university/Apartments in Germany/clean_data_y.csv')
clean_data_y.head()

,Unnamed: 0,livingSpace
0,0,86.00
1,1,89.00
2,2,83.80
3,3,58.15
4,4,84.97


## Train Test split ##

In [32]:
train_size = int(0.8 * clean_data_x.shape[0])
train_size

200380

In [33]:
x_train = clean_data_x[:train_size]
y_train = clean_data_y[:train_size]

print(x_train.shape)
print(y_train.shape)

x_test = clean_data_x[train_size:]
y_test = clean_data_y[train_size:]

print(x_test.shape)
print(y_test.shape)

(200380, 211)
(200380, 2)
(50096, 211)
(50096, 2)


## linear regression ##

In [35]:
from sklearn.metrics import mean_squared_error

class LinearRegression:

  def __init__(self, learning_rate=0.1, epochs=100, accuracy_rate=0.1):
    self.learning_rate = learning_rate
    self.epochs = epochs
    self.accuracy_rate = accuracy_rate
    self.history = {'loss': [], 'accuracy': []}

  def gradient(self, X, y):
    return 2/X.shape[0] * np.dot(X.T, (np.dot(X, self.weights) - y))

  def fit(self, X, y, Xval, yval):
    train = []
    for i in range(len(X)):
      x = list(X[i])
      x.insert(0, 1)
      train.append(np.array(x))
    train = np.array(train)
    self.weights = np.random.rand((train.shape[1]))
    for i in range(self.epochs + 1):
      pred = self.predict(Xval)
      err = mean_squared_error(yval, pred)
      acc = self.calculate_accuracy(pred, yval)
      self.history['loss'].append(err)
      self.history['accuracy'].append(acc)
      self.weights = self.weights - self.learning_rate * self.gradient(train, y)

  def predict(self, X):
    pred = []
    for i in range(len(X)):
      x = list(X[i])
      x.insert(0, 1)
      pred.append(np.array(x))
    return np.dot(pred, self.weights)

  def calculate_accuracy(self, pred, val):
    right = 0
    for i in range(len(pred)):
      if abs(pred[i] - val[i]) <= val[i] * self.accuracy_rate:
        right += 1
    return right / len(pred)

## k_fold ##

In [36]:
from sklearn.utils import shuffle

def k_fold(Model, X, y, k=10, epochs=50):
  X, y = shuffle(X, y, random_state=0)
  n = len(y)
  X = np.array([X]).T
  y = np.array(y)
  folds_acc = []
  sum = 0
  j = 1
  for i in range(0, n, n//k):
    trainx = np.concatenate((X[0:i], X[i + n//k: n]), axis=0)
    trainy = np.concatenate((y[0:i], y[i + n//k: n]), axis=0)
    valx = X[i: i + n//k]
    valy = y[i: i + n//k]
    model = Model(epochs=epochs)
    model.fit(trainx, trainy, valx, valy)
    folds_acc.append(model.history['accuracy'][-1])
    print('fold', j, ':', model.history['accuracy'][-1])
    sum += model.history['accuracy'][-1]
    j += 1
  return sum/k, folds_acc

## most correlated feature regression ##

In [37]:
x = x_train['livingSpaceRange']
y = y_train['livingSpace']
print('5-fold:')
acc_5_fold, acc_list_5_fold = k_fold(LinearRegression, x, y, k=5)
print ('accuracy:', acc_5_fold)

print()

print('10-fold:')
acc_10_fold, acc_list_10_fold = k_fold(LinearRegression, x, y, k=10)
print ('accuracy:', acc_10_fold)

5-fold:
fold 1 : 0.5214093222876535
fold 2 : 0.5242289649665636
fold 3 : 0.5978391056991715
fold 4 : 0.517491765645274
fold 5 : 0.5242788701467213
accuracy: 0.5370496057490768

10-fold:
fold 1 : 0.5291945303922547
fold 2 : 0.5328875137239245
fold 3 : 0.5307415909771435
fold 4 : 0.528146521608943
fold 5 : 0.5724124164088232
fold 6 : 0.5639784409621719
fold 7 : 0.5264497454835811
fold 8 : 0.5260005988621619
fold 9 : 0.5317396945802975
fold 10 : 0.5287453837708355
accuracy: 0.5370296436770137


## k_fold with pakages ##

In [38]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [40]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)
model = LinearRegression()
scores = cross_val_score(model, x_train['livingSpaceRange'].values.reshape(-1, 1), y_train['livingSpace']
                         , scoring='neg_mean_squared_error', cv=cv)
print(scores)

[-9.15780369e+01 -2.19403221e+05 -9.76805106e+02 -6.14411694e+05
 -1.06401333e+02 -3.71957236e+03 -1.10673699e+02 -1.07076041e+02
 -2.34871996e+02 -9.46926305e+01]


In [42]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)
model = LinearRegression()
scores = cross_val_score(model, x_train[['livingSpaceRange', 'pricetrend', 'thermalChar', 'floor']], y_train['livingSpace']
                         , scoring='neg_mean_squared_error', cv=cv)
print(scores)

[-9.18390411e+01 -2.19401704e+05 -9.77462137e+02 -6.14410151e+05
 -1.06644331e+02 -3.71968867e+03 -1.10758107e+02 -1.07287904e+02
 -2.35366496e+02 -9.50846027e+01]


In [43]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)
model = LinearRegression()
scores = cross_val_score(model, x_train, y_train['livingSpace']
                         , scoring='neg_mean_squared_error', cv=cv)
print(scores)

[-1.19956584e+02 -2.19417405e+05 -9.94840463e+02 -6.14468916e+05
 -1.29051674e+02 -3.73476724e+03 -1.33158944e+02 -1.33394651e+02
 -2.60580804e+02 -1.22743194e+02]


## Ridge and Lasso ##

In [45]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [51]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)
model = Ridge()
scores = cross_val_score(model, x_train, y_train['livingSpace'], scoring='neg_mean_squared_error', cv=cv)
print(scores)

[-1.20018641e+02 -2.19416678e+05 -9.94773279e+02 -6.14467954e+05
 -1.29273385e+02 -3.73481680e+03 -1.33324155e+02 -1.33148380e+02
 -2.60635011e+02 -1.22763347e+02]


In [52]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)
model = Lasso()
scores = cross_val_score(model, x_train, y_train['livingSpace'], scoring='neg_mean_squared_error', cv=cv)
print(scores)

[-8.64977508e+01 -2.19417812e+05 -9.73699754e+02 -6.14438116e+05
 -1.01990843e+02 -3.71868310e+03 -1.07178888e+02 -1.02551562e+02
 -2.31626666e+02 -8.99023678e+01]
